# Uniswap V2 Tutorial

This tutorial shows how to use Dexsnake to use Uniswap V2. Let's start by connecting to the blockchain (the code snippet below assumes you have configured the required environment variables):

In [12]:
import os

from web3 import Web3


account = os.getenv("WEB3_PUBLIC_KEY")
private_key = os.getenv("WEB3_PRIVATE_KEY")
provider_url = os.getenv("WEB3_PROVIDER_URL")

web3 = Web3(Web3.HTTPProvider(provider_url))

Now, let's import `UniswapV2Factory` and find the pair with USDC and WETH.

In [2]:
from dexsnake.uniswap_v2 import UniswapV2Factory


factory = UniswapV2Factory(web3)

# These are the addresses of USDC and WETH on Base. If you are using a Web3
# instance connected to a different blockchain, update these.
USDC = "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913"
WETH = "0x4200000000000000000000000000000000000006"

pair_address = factory.get_pair(USDC, WETH)

print(pair_address)

0x88A43bbDF9D098eEC7bCEda4e2494615dfD9bB9C


Now, let's initialize the pair object, fetch the reserves of the pair and calculate the price of WETH using the constant product formula:

In [3]:
from dexsnake.uniswap_v2 import UniswapV2Pair


pair = UniswapV2Pair(web3, pair_address)

reserves_0, reserves_1, timestamp_last_updated = pair.get_reserves()

print(reserves_0 / 1e18)  # WETH reserves
print(reserves_1 / 1e6)  # USDC reserves

price = reserves_1 / reserves_0

print(price * 1e18 / 1e6)  # price of WETH after taking into account the decimals

494.5171313913738
1214088.117786
2455.0981972455443


Now that we know the pair has good liquidity at a good price, let's use the router to swap one USDC for WETH (this assumes that you have already approved the router contract to spend your USDC).

In [11]:
from dexsnake.uniswap_v2 import UniswapV2Router


router = UniswapV2Router(web3)

tx_receipt = router.swap_exact_tokens_for_tokens(
    amount_in=int(1 * 1e6),
    amount_out_min=int(1 * 1e6 / price * 0.99),
    path=[USDC, WETH],
    to=account,
    account=account,
    private_key=private_key,
)

tx_receipt

AttributeDict({'blockHash': HexBytes('0xa858551a1f9dec6a24c04103074606210dbccafe6bdd028e4baaab5c395f3531'),
 'blockNumber': 18164904,
 'contractAddress': None,
 'cumulativeGasUsed': 18413451,
 'effectiveGasPrice': 8093245,
 'from': '0x011b7DD4cAA8E48FbBaef548dB349dfB3D09aA3F',
 'gasUsed': 127659,
 'l1BaseFeeScalar': '0x8dd',
 'l1BlobBaseFee': '0x1',
 'l1BlobBaseFeeScalar': '0x101c12',
 'l1Fee': '0x909580b25',
 'l1GasPrice': '0x1b8183215',
 'l1GasUsed': '0x90c',
 'logs': [AttributeDict({'address': '0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x000000000000000000000000011b7dd4caa8e48fbbaef548db349dfb3d09aa3f'),
    HexBytes('0x00000000000000000000000088a43bbdf9d098eec7bceda4e2494615dfd9bb9c')],
   'data': HexBytes('0x00000000000000000000000000000000000000000000000000000000000f4240'),
   'blockNumber': 18164904,
   'transactionHash': HexBytes('0x0d46f872967e2e8be4c2492bdc5c8abe442b

You can inspect the transaction on the block explorer: https://basescan.org/tx/0x0d46f872967e2e8be4c2492bdc5c8abe442b0ade98bec51b2e4c61d280258b7f.